In [557]:
import pandas as pd
from sklearn.linear_model import LogisticRegression

In [558]:
database = pd.read_csv('COBRA-YTD2017.csv')

In [559]:
database.rename(columns={'UC2 Literal' : 'Crime', 'occur_time': 'time', 'MaxOfnum_victims': 'victims', 'Avg Day': 'day'}, inplace=True)
# database.head()
database.Crime.value_counts()

LARCENY-FROM VEHICLE    9840
LARCENY-NON VEHICLE     6589
AUTO THEFT              3197
BURGLARY-RESIDENCE      2635
AGG ASSAULT             2024
ROBBERY-PEDESTRIAN      1126
BURGLARY-NONRES          758
RAPE                     226
ROBBERY-COMMERCIAL       157
ROBBERY-RESIDENCE        132
HOMICIDE                  75
Name: Crime, dtype: int64

In [560]:
# NOTE : I removed rows where victims are only 1, it screws up the results.

# database.day.unique()
arr = [23, 24, 21, 19, 27, 17, 15]
database = database[database.victims != 23]
database = database[database.victims != 24]
database = database[database.victims != 21]
database = database[database.victims != 19]
database = database[database.victims != 27]
database = database[database.victims != 17]
database = database[database.victims != 15]
database = database[database.Crime != 'HOMICIDE']
database.Crime.value_counts()

LARCENY-FROM VEHICLE    9837
LARCENY-NON VEHICLE     6586
AUTO THEFT              3197
BURGLARY-RESIDENCE      2635
AGG ASSAULT             2023
ROBBERY-PEDESTRIAN      1126
BURGLARY-NONRES          758
RAPE                     226
ROBBERY-COMMERCIAL       157
ROBBERY-RESIDENCE        132
Name: Crime, dtype: int64

In [561]:
# NOTE: Clean up database and remove strings basically.

database['Crime'] = database['Crime'].replace({'LARCENY-FROM VEHICLE': '1',
                          'ROBBERY-PEDESTRIAN': '2',
                          'BURGLARY-RESIDENCE': '3',
                          'AUTO THEFT': '4',
                          'BURGLARY-NONRES': '5',
                          'ROBBERY-RESIDENCE': '6',
                          'LARCENY-NON VEHICLE': '7',
                          'AGG ASSAULT': '8',
                            'RAPE': '9',
                          'ROBBERY-COMMERCIAL': '10',
                          'HOMICIDE': '11'})

database['day'] = database['day'].replace({'Mon': '1',
                                          'Tue': '2',
                                          'Wed': '3',
                                          'Thu': '4',
                                          'Fri': '5',
                                          'Sat': '6',
                                          'Sun': '7',
                                          'Unk': '0'})

database.Crime.nunique()

10

In [562]:
# NOTE : Gotta convert everything to int for train_test_split

database['time'] = pd.to_datetime(database.time)
# # TODO: Figure out how to plot a scatter_matrix

# # from pandas.plotting import scatter_matrix

# # database['occur_time'] = pd.to_datetime(database['occur_time'], errors='coerce')

# # database['occur_time'] = database['occur_time'].dt.time

database['hour'] = database.time.dt.hour
# database['hour'].value_counts()

database['victims'] = database['victims'].fillna(0)
database.victims = database.victims.astype(int)
database.hour = database.victims.astype(int)
database.x = database.x.astype(int)
database.y = database.y.astype(int)
database.day = database.day.astype(int)

# database.dtypes

In [563]:
from sklearn.preprocessing import StandardScaler
from sklearn import tree

# database = database[database.Crime != 1]

scaler = StandardScaler()
X_all = database.drop(['Crime'], 1)
y_all = database['Crime']

X_all = database[['hour', 'day', 'victims']]

X_all = X_all.fillna(0)

X_all = scaler.fit_transform(X_all)

In [564]:
clf = tree.DecisionTreeClassifier()

In [565]:
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, random_state = 2)

In [566]:
from sklearn.datasets import make_multilabel_classification
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC

In [567]:
classif = OneVsRestClassifier(SVC(kernel = 'rbf'))
classif.fit(X_train, y_train)

y_pred = classif.predict(X_train)

print(sum(y_train == y_pred) / float(len(y_pred)))

y_pred = classif.predict(X_test)

print(sum(y_test == y_pred) / float(len(y_pred)))

0.22372169740590794
0.2143928035982009


In [ ]:
auto : 28%
rape : 24%
burglary-res : 24%
larch-nonveh : 23
robbpeds 22
burgnonr 21
r